In [2]:
%load_ext rpy2.ipython
%load_ext autotime
import rpy2.robjects.lib.ggplot2 as ggplot2
import childespy
import numpy as np
import os
import sys
import imp
import pandas as pd
import transformers
import torch
import re
import unicodedata
import scipy.stats
import copy
from string import punctuation
from pprint import pprint

R[write to console]: Installing package into ‘/home/stephan/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)



Reinstalling childesr version 0.2.1


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/childesr_0.2.3.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 22865 bytes (22 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to conso

time: 10.6 s (started: 2022-04-26 18:32:55 -07:00)


In [3]:
%%R
# load R dependencies
library('plyr')
library('tidyr')
library('plotROC')
library('pROC')

R[write to console]: Type 'citation("pROC")' for a citation.

R[write to console]: 
Attaching package: ‘pROC’


R[write to console]: The following object is masked from ‘package:plotROC’:

    ggroc


R[write to console]: The following objects are masked from ‘package:stats’:

    cov, smooth, var




time: 192 ms (started: 2022-04-26 18:33:06 -07:00)


In [4]:
sys.path.append('../../')
from src.utils import sample_models_across_time, load_models, hyperparameter_utils, configuration, paths
config = configuration.Config()

time: 256 ms (started: 2022-04-26 18:33:21 -07:00)


In [5]:
%%R
filter_model_subset = function(this_df, model_names_list){
    #return(this_df[this_df$model %in% model_names_list])
    return(subset(this_df, this_df$model %in% model_names_list))
}

sem <- function(x) sd(x)/sqrt(length(x))

time: 8.79 ms (started: 2022-04-26 18:33:29 -07:00)


# Load Scores and move to R

In [79]:
import imp
imp.reload(sample_models_across_time)
imp.reload(load_models)
imp.reload(hyperparameter_utils)
imp.reload(paths)
config = configuration.Config()

time: 9.6 ms (started: 2022-04-26 20:26:37 -07:00)


In [80]:
levdist_scores_across_time = pd.concat(sample_models_across_time.assemble_scores_no_order('levdist'))
# Remove 0.5 and 4.0 because they are sparse.
levdist_scores_across_time = levdist_scores_across_time[~levdist_scores_across_time.age.isin({0.5, 4.0})]
levdist_scores_across_time['likelihood_type'] = 'Levenshtein Distance'

Searching /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/eval/n=5000/Providence_all_with_tagsxBERT_Providence_all_0/levdist_run_models_across_time_*.pkl
Searching /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/eval/n=5000/Providence_all_with_tagsxBERT_Providence_all_20/levdist_run_models_across_time_*.pkl
Searching /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/eval/n=5000/Providence_all_no_tagsxBERT_Providence_all_0/levdist_run_models_across_time_*.pkl
Searching /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/eval/n=5000/Providence_all_no_tagsxBERT_Providence_all_20/levdist_run_models_across_time_*.pkl
Searching /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/eval/n=5000/Providence-Age_young_with_tagsxBERT_Providence_all_0/levdist_run_models_across_time_*.pkl
Searching /home/stephan/notebooks/child-directed-listening/output/exper

In [81]:
np.unique(levdist_scores_across_time.model)

array(['Providence-Age_oldxno_tagsxBERTxProvidence_all_Providence-Age_0',
       'Providence-Age_oldxno_tagsxBERTxProvidence_all_Providence-Age_20',
       'Providence-Age_oldxwith_tagsxBERTxProvidence_all_Providence-Age_0',
       'Providence-Age_oldxwith_tagsxBERTxProvidence_all_Providence-Age_20',
       'Providence-Age_youngxno_tagsxBERTxProvidence_all_Providence-Age_0',
       'Providence-Age_youngxno_tagsxBERTxProvidence_all_Providence-Age_20',
       'Providence-Age_youngxwith_tagsxBERTxProvidence_all_Providence-Age_0',
       'Providence-Age_youngxwith_tagsxBERTxProvidence_all_Providence-Age_20',
       'Providence_allxno_tagsxBERTxProvidence_all_Providence_0',
       'Providence_allxno_tagsxBERTxProvidence_all_Providence_20',
       'Providence_allxno_tagsxdata_unigramxProvidence_all_Providence_0',
       'Providence_allxwith_tagsxBERTxProvidence_all_Providence_0',
       'Providence_allxwith_tagsxBERTxProvidence_all_Providence_20',
       'Switchboard_allxno_tagsxBERTxProvide

time: 6.15 ms (started: 2022-04-26 20:26:43 -07:00)


In [82]:
wfst_scores_across_time = pd.concat(sample_models_across_time.assemble_scores_no_order('wfst'))
# Remove 0.5 and 4.0 because they are sparse.
wfst_scores_across_time = wfst_scores_across_time[~wfst_scores_across_time.age.isin({0.5, 4.0})]
wfst_scores_across_time['likelihood_type'] = 'WFST'

Searching /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/eval/n=5000/Providence_all_with_tagsxBERT_Providence_all_0/wfst_run_models_across_time_*.pkl
Searching /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/eval/n=5000/Providence_all_with_tagsxBERT_Providence_all_20/wfst_run_models_across_time_*.pkl
Searching /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/eval/n=5000/Providence_all_no_tagsxBERT_Providence_all_0/wfst_run_models_across_time_*.pkl
Searching /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/eval/n=5000/Providence_all_no_tagsxBERT_Providence_all_20/wfst_run_models_across_time_*.pkl
Searching /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/eval/n=5000/Providence-Age_young_with_tagsxBERT_Providence_all_0/wfst_run_models_across_time_*.pkl
Searching /home/stephan/notebooks/child-directed-listening/output/experiments/full_sca

In [83]:
np.unique(wfst_scores_across_time.model)

array(['Providence-Age_oldxno_tagsxBERTxProvidence_all_Providence-Age_0',
       'Providence-Age_oldxno_tagsxBERTxProvidence_all_Providence-Age_20',
       'Providence-Age_oldxwith_tagsxBERTxProvidence_all_Providence-Age_0',
       'Providence-Age_oldxwith_tagsxBERTxProvidence_all_Providence-Age_20',
       'Providence-Age_youngxno_tagsxBERTxProvidence_all_Providence-Age_0',
       'Providence-Age_youngxno_tagsxBERTxProvidence_all_Providence-Age_20',
       'Providence-Age_youngxwith_tagsxBERTxProvidence_all_Providence-Age_0',
       'Providence-Age_youngxwith_tagsxBERTxProvidence_all_Providence-Age_20',
       'Providence_allxno_tagsxBERTxProvidence_all_Providence_0',
       'Providence_allxno_tagsxBERTxProvidence_all_Providence_20',
       'Providence_allxno_tagsxdata_unigramxProvidence_all_Providence_0',
       'Providence_allxwith_tagsxBERTxProvidence_all_Providence_0',
       'Providence_allxwith_tagsxBERTxProvidence_all_Providence_20',
       'Switchboard_allxno_tagsxBERTxProvide

time: 6.58 ms (started: 2022-04-26 20:27:11 -07:00)


In [84]:
scores_across_time = pd.concat([levdist_scores_across_time, wfst_scores_across_time])

time: 17.6 ms (started: 2022-04-26 20:27:15 -07:00)


In [104]:
# this takes forever -- moving things into R is expensive
scores_across_time['token'] = scores_across_time['token'].astype(str)
scores_across_time_short = scores_across_time[['posterior_probability','posterior_entropy','prior_probability','prior_entropy',
    'kl_flat_to_prior', 'kl_flat_to_posterior', 'model','set','age','token','bert_token_id', 'edit_distance','likelihood_type','prior_rank','posterior_rank']]
%R -i scores_across_time_short 

time: 380 ms (started: 2022-04-26 22:08:25 -07:00)


In [105]:
%%R
levels(factor(scores_across_time_short$model))

 [1] "adult-written_allxno_tagsxBERTxProvidence_all_adult-written_0"       
 [2] "adult-written_allxno_tagsxBERTxProvidence_all_adult-written_20"      
 [3] "no-split_no-datasetxno_tagsxflat_unigramxProvidence_all_no-split_0"  
 [4] "Providence_allxno_tagsxBERTxProvidence_all_Providence_0"             
 [5] "Providence_allxno_tagsxBERTxProvidence_all_Providence_20"            
 [6] "Providence_allxno_tagsxdata_unigramxProvidence_all_Providence_0"     
 [7] "Providence_allxwith_tagsxBERTxProvidence_all_Providence_0"           
 [8] "Providence_allxwith_tagsxBERTxProvidence_all_Providence_20"          
 [9] "Providence-Age_oldxno_tagsxBERTxProvidence_all_Providence-Age_0"     
[10] "Providence-Age_oldxno_tagsxBERTxProvidence_all_Providence-Age_20"    
[11] "Providence-Age_oldxwith_tagsxBERTxProvidence_all_Providence-Age_0"   
[12] "Providence-Age_oldxwith_tagsxBERTxProvidence_all_Providence-Age_20"  
[13] "Providence-Age_youngxno_tagsxBERTxProvidence_all_Providence-Age_0"   
[14] "Provid

In [106]:
%%R

model_remapper = list()
model_remapper[["adult-written_allxno_tagsxBERTxProvidence_all_adult-written_0"]] = 'BERT+OneUtt'
model_remapper[["adult-written_allxno_tagsxBERTxProvidence_all_adult-written_20"]] = 'BERT+Context'      
model_remapper[["no-split_no-datasetxno_tagsxflat_unigramxProvidence_all_no-split_0" ]] = 'UniformPrior'
model_remapper[["Providence_allxno_tagsxBERTxProvidence_all_Providence_0"]]  = 'CDL+OneUtt'
model_remapper[["Providence_allxno_tagsxBERTxProvidence_all_Providence_20"]] = 'CDL+Context'
model_remapper[["Providence_allxno_tagsxdata_unigramxProvidence_all_Providence_0"]] = 'CHILDES 1gram'     
model_remapper[["Providence_allxwith_tagsxBERTxProvidence_all_Providence_0"]] = 'CDL+OneUtt+Tags'
model_remapper[["Providence_allxwith_tagsxBERTxProvidence_all_Providence_20"]] = 'CDL+Context+Tags'
model_remapper[["Providence-Age_oldxno_tagsxBERTxProvidence_all_Providence-Age_0"]] = 'CDL+Older+OneUtt'     
model_remapper[["Providence-Age_oldxno_tagsxBERTxProvidence_all_Providence-Age_20"]] = 'CDL+Older+Context'
model_remapper[["Providence-Age_oldxwith_tagsxBERTxProvidence_all_Providence-Age_0"]] = 'CDL+Older+OneUtt+Tags'
model_remapper[["Providence-Age_oldxwith_tagsxBERTxProvidence_all_Providence-Age_20"]] = 'CDL+Older+Context+Tags'
model_remapper[["Providence-Age_youngxno_tagsxBERTxProvidence_all_Providence-Age_0"]] = 'CDL+Younger+OneUtt'     
model_remapper[["Providence-Age_youngxno_tagsxBERTxProvidence_all_Providence-Age_20"]] = 'CDL+Younger+Context'
model_remapper[["Providence-Age_youngxwith_tagsxBERTxProvidence_all_Providence-Age_0"]] = 'CDL+Younger+OneUtt+Tags'
model_remapper[["Providence-Age_youngxwith_tagsxBERTxProvidence_all_Providence-Age_20"]] = 'CDL+Younger+Context+Tags'
model_remapper[["Switchboard_allxno_tagsxBERTxProvidence_all_Switchboard_0"]] = 'Switchboard+OneUtt'
model_remapper[["Switchboard_allxno_tagsxBERTxProvidence_all_Switchboard_20"]] = 'Switchboard+Context+Tags'  

time: 8.72 ms (started: 2022-04-26 22:08:32 -07:00)


In [107]:
%%R
scores_across_time_short$model = sapply(scores_across_time_short$model, function(x){model_remapper[[x]]})

time: 80.4 ms (started: 2022-04-26 22:08:36 -07:00)


In [108]:
%%R

successes_across_time = subset(scores_across_time_short, set == 'success')

time: 196 ms (started: 2022-04-26 22:08:40 -07:00)


In [109]:
%%R 
get_ref_df = function(model, single_df){
    rdf = single_df
    single_df$line_model = single_df$model
    single_df$model = model
    return(single_df) 
}

time: 10 ms (started: 2022-04-26 22:08:42 -07:00)


In [110]:
%%R
overview_set = c(
    'CDL+Context+Tags',
    'BERT+Context',
    'CDL+OneUtt+Tags',
    'BERT+OneUtt',
    'Switchboard+Context',
    'Switchboard+OneUtt',
    'CHILDES 1gram',
    'UniformPrior'
)

splits_tags_set <-  c(
    "CDL+Context+Tags",
    "CDL+Context",
    "CDL+Older+Context+Tags",
    "CDL+Older+Context",
    "CDL+Younger+Context+Tags",
    "CDL+Younger+Context"
)

time: 7.78 ms (started: 2022-04-26 22:08:45 -07:00)


# Verify Sample Size

In [111]:
%%R
successes_and_failures_per_time = aggregate(bert_token_id ~ set + age, subset(scores_across_time_short, likelihood_type == 'WFST' & model == 'UniformPrior'), 
          length)
print(successes_and_failures_per_time)

print(sum(successes_and_failures_per_time$bert_token_id))

       set age bert_token_id
1  failure 1.0            10
2  success 1.0            12
3  failure 1.5            10
4  success 1.5            14
5  failure 2.0            10
6  success 2.0            30
7  failure 2.5            10
8  success 2.5            40
9  failure 3.0            10
10 success 3.0            22
11 failure 3.5            10
12 success 3.5            40
[1] 218
time: 63 ms (started: 2022-04-26 22:08:48 -07:00)


# Analysis 1: Is the child's vocalization intelligible? 

## ROC for Whether a Word Was Found

In [ ]:
%%R 
scores_across_time_short$set_binarized = 0
scores_across_time_short$set_binarized[scores_across_time_short$set == 'failure'] = 1
table(scores_across_time_short$set_binarized)

In [ ]:
%%R
# Get AUC scores into these
unique(as.character(scores_across_time_short$model))

## WFST


In [ ]:
%%R
auc_by_model = do.call('rbind', lapply(unique(subset(scores_across_time_short, likelihood_type == 'WFST')$model), 
      function(model_name){
    one_model = subset(scores_across_time_short, model == model_name & likelihood_type == 'WFST')
    roc1 = roc(one_model$set_binarized, one_model$posterior_entropy)
    return(data.frame(model=model_name, auc_wfst = auc(roc1)))}))

scores_across_time_short_wfst = merge(subset(scores_across_time_short, likelihood_type == 'WFST') , auc_by_model)



In [ ]:
%%R

# get rid of tags in the designation
scores_across_time_overview_set = filter_model_subset(scores_across_time_short_wfst, overview_set)
scores_across_time_overview_set$model_shortname = gsub('\\+Tags','', scores_across_time_overview_set$model)

scores_across_time_overview_set$model_shortname_with_wfst_auc = paste0(scores_across_time_overview_set$model_shortname,' (AUC = ',round(scores_across_time_overview_set$auc_wfst, 3),')')

In [ ]:
%%R 
# get the scores associated with each model + a  
names_per_model = aggregate(model_shortname_with_wfst_auc ~ model_shortname, scores_across_time_overview_set, unique)
scores_per_model_wfst = aggregate(auc_wfst ~ model_shortname,scores_across_time_overview_set, unique)
scores_per_model_wfst = merge(names_per_model, scores_per_model_wfst)
scores_per_model_wfst = scores_per_model_wfst[order(scores_per_model_wfst$auc_wfst, decreasing=T),]
print(scores_per_model_wfst)

scores_across_time_overview_set$model_shortname_with_wfst_auc = factor(as.character(scores_across_time_overview_set$model_shortname_with_wfst_auc), levels = as.character(scores_per_model_wfst$model_shortname_with_wfst_auc))

In [ ]:
%%R -h 700 -w 700
p1 = ggplot(scores_across_time_overview_set, 
       aes(d=set_binarized, m=posterior_entropy, color=model_shortname_with_wfst_auc)
) + theme_classic(base_size=28)+ geom_roc(n.cuts=0) + xlab('Proportion of False Positives'
) + ylab('Proportion of True Positives') + theme(legend.position=c(.67, .20),
legend.background = element_blank(), legend.title = element_blank()) + annotate("text",
label= "Higher is better", size=8, fontface="italic", x=.15, y=1, alpha =.5) + geom_abline(intercept=0, slope=1)
print(p1)
ggsave('../../output/figures/roc_wfst.pdf', height=10, width=10)    

### Set Color Standards for All Plots

In [ ]:
%%R
g <- ggplot_build(p1)
colors_for_8_plots = unique(g$data[[1]]$colour)
reorder = c(1,3,2,4,5,6,7,8)
colors_for_8_plots = colors_for_8_plots[reorder]
print(colors_for_8_plots)

In [ ]:
%%R
models_for_8_plots = scores_per_model$model_shortname[reorder]
print(as.character(models_for_8_plots))
shapes_for_8_plots = c(19, 17,  7, 18, 4, 15, 3, 1)

In [ ]:
scores_across_time_short_lev$model_shortname= gsub('\\+Tags','',scores_across_time_short_lev$model)
scores_across_time_short_lev$model_shortname = factor(scores_across_time_short_lev$model_shortname, levels = models_for_8_plots)

In [ ]:
%%R -h 700 -w 700
p1 = ggplot(scores_across_time_overview_set, 
       aes(d=set_binarized, m=posterior_entropy, color=model_shortname_with_wfst_auc)
) + theme_classic(base_size=28)+ geom_roc(n.cuts=0) + xlab('Proportion of False Positives'
) + ylab('Proportion of True Positives') + theme(legend.position=c(.67, .20),
legend.background = element_blank(), legend.title = element_blank()) + annotate("text",
label= "Higher is better", size=8, fontface="italic", x=.15, y=1, alpha =.5) + geom_abline(intercept=0, slope=1)
#ggsave('../../output/figures/roc_wfst.pdf', height=10, width=10)    

In [ ]:
%%R -h 700 -w 700
p1 = ggplot(scores_across_time_overview_set, 
       aes(d=set_binarized, m=posterior_entropy, color=model_shortname_with_wfst_auc)
) + theme_classic(base_size=28)+ geom_roc(n.cuts=0) + xlab('Proportion of False Positives'
) + ylab('Proportion of True Positives') + theme(legend.position=c(.67, .20),
legend.background = element_blank(), legend.title = element_blank()) + annotate("text",
label= "Higher is better", size=8, fontface="italic", x=.15, y=1, alpha =.5) + geom_abline(intercept=0, slope=1)
print(p1)
ggsave('../../output/figures/roc_wfst.pdf', height=10, width=10)    

## Levenshtein Distance

In [ ]:
%%R
auc_by_model_lev = do.call('rbind', lapply(unique(subset(scores_across_time_short, likelihood_type == 'Levenshtein Distance')$model), 
      function(model_name){
    one_model = subset(scores_across_time_short, model == model_name & likelihood_type == 'Levenshtein Distance')
    roc1 = roc(one_model$set_binarized, one_model$posterior_entropy)
    return(data.frame(model=model_name, auc_levdist = auc(roc1)))}))

scores_across_time_short_lev = merge(subset(scores_across_time_short, likelihood_type == 'Levenshtein Distance') , auc_by_model_lev)

In [ ]:
%%R
scores_across_time_short_lev$model_with_levdist_auc = paste0(scores_across_time_short_lev$model,' (AUC = ',
round(scores_across_time_short_lev$auc_levdist, 3),')')

In [ ]:
%%R
# sort the levels by the scores
names_per_model = aggregate(model_with_levdist_auc ~ model, scores_across_time_short_lev, unique)
scores_per_model_lev = aggregate(auc_levdist ~ model, scores_across_time_short_lev, unique)
scores_per_model_lev = merge(names_per_model, scores_per_model_lev)
scores_per_model_lev = scores_per_model_lev[order(scores_per_model_lev$auc_levdist, decreasing=T),]
print(scores_per_model_lev)
scores_across_time_short_lev$model_with_levdist_auc = factor(as.character(scores_across_time_short_lev$model_with_levdist_auc), 
                                                             levels = as.character(scores_per_model_lev$model_with_levdist_auc))

In [ ]:
%%R
# fix the model titles and the line colors
scores_across_time_short_lev$model_shortname= gsub('\\+Tags','',scores_across_time_short_lev$model)
scores_across_time_short_lev$model_shortname = factor(scores_across_time_short_lev$model_shortname, levels = models_for_8_plots)


scores_across_time_short_wfst$model_shortname= gsub('\\+Tags','',scores_across_time_short_wfst$model)
scores_across_time_short_wfst$model_shortname = factor(scores_across_time_short_wfst$model_shortname, levels = models_for_8_plots)

In [ ]:
%%R -h 700 -w 700
p1 = ggplot(filter_model_subset(scores_across_time_short_lev, overview_set), 
       aes(d=set_binarized, m=posterior_entropy, color=model_shortname), linetype = 'dashed'
) + theme_classic(base_size=28)+ geom_roc(n.cuts=0, linetype="dotted") + xlab('Proportion of False Positives'
) + ylab('Proportion of True Positives') + theme(legend.position="none")  + geom_abline(intercept=0, slope=1
) + facet_wrap(~model_shortname) + geom_roc(data=filter_model_subset(scores_across_time_short_wfst,
overview_set), aes(d=set_binarized, 
m=posterior_entropy, color=model_shortname), n.cuts=0) + theme(strip.text.x = element_text(size = 20))
ggsave('../../output/figures/roc_levdist.pdf', height=10, width=10)   + scale_color_manual(
        values=colors_for_8_plots)  
print(p1)

In [ ]:
%%R -h 700 -w 700
p1 = ggplot(filter_model_subset(scores_across_time_short_lev, overview_set), 
       aes(d=set_binarized, m=posterior_entropy, color=model_shortname), linetype = 'dashed'
) + theme_classic(base_size=28)+ geom_roc(n.cuts=0, linetype="dotted") + xlab('Proportion of False Positives'
) + ylab('Proportion of True Positives') + geom_abline(intercept=0, slope=1
) + facet_wrap(~model_shortname) + geom_roc(data=filter_model_subset(scores_across_time_short_wfst,
overview_set), aes(d=set_binarized,  m=posterior_entropy, color=model_shortname), n.cuts=0) + theme(strip.text.x = element_text(size = 20)
) +  guides(color = guide_legend(reverse=TRUE))
print(p1)

## Visualization of Successes and Failures

In [ ]:
%%R
unique(as.character(scores_across_time_short$model))

In [ ]:
%%R
names(scores_across_time_short)

In [ ]:
%%R
index_cols = c('bert_token_id', 'set')
selected_cols = c('posterior_entropy', 'posterior_probability', 'posterior_rank', 'prior_entropy', 'prior_probability', 'prior_rank')
best_model = subset(scores_across_time_short, model == "CDL+Context+Tags" & likelihood_type == 'WFST')[,c(index_cols,selected_cols)]
names(best_model) = c(index_cols, paste0('bert_', selected_cols))
base_model = subset(scores_across_time_short, model == "UniformPrior" & likelihood_type == 'WFST')[,c(index_cols,selected_cols)]
names(base_model) = c(index_cols, paste0('uniform_', selected_cols))
model_scores_merged = merge(best_model, base_model)

In [ ]:
%%R -w 750 -h 500 
ggplot(model_scores_merged) + geom_point(aes(x=uniform_posterior_entropy, y=bert_posterior_entropy, color=set), alpha=.25, size=1
) + theme_classic(base_size=28) + xlab('Posterior Entropy Under A Uniform Prior') + ylab('Posterior Entropy Under CDL Prior')

## Statistical Tests for Difference in ROC curves

### Pairwise comparison of WFST ROC curves

In [ ]:
%%R

models = unique(subset(scores_across_time_short, likelihood_type == 'WFST')$model)
models[1]
roc_tests = mat.or.vec(length(models), length(models))

In [ ]:
%%R



#precompute ROCs
rocs = list()
for (model_name in overview_set){
    one_model = subset(scores_across_time_short, model == model_name & likelihood_type == 'WFST')
    rocs[[model_name]] = roc(one_model$set_binarized, one_model$posterior_entropy)
}

In [ ]:
%%R

roc_ps = mat.or.vec(length(overview_set), length(overview_set))
roc_stats = mat.or.vec(length(overview_set), length(overview_set))



for (i in c(1:length(overview_set))){
    for (j in c(1:i)){
        roc1 = rocs[[overview_set[i]]]
        roc2 = rocs[[overview_set[j]]]
        rtest = roc.test(roc1, roc2, method="delong")
        roc_ps[i,j] = round(rtest$p.value, 3)        
        roc_stats [i,j] = abs(round(rtest$statistic, 3))        
    }
}    


roc_ps = data.frame(roc_ps)
names(roc_ps) = overview_set
colnames(roc_ps) = overview_set


roc_stats = data.frame(roc_stats)
names(roc_stats) = overview_set
colnames(roc_stats) = overview_set

In [ ]:
%%R

roc_stats

In [ ]:
%%R
roc_ps

In [ ]:
%%R
roc_stats[5,2] = NA
print(min(roc_stats[roc_stats > 0], na.rm=T))
print(max(roc_stats[roc_stats > 0], na.rm=T))

In [ ]:
%%R
roc_ps[5,2] = NA
print(min(roc_ps[roc_ps < 1], na.rm=T))
print(max(roc_ps[roc_ps < 1], na.rm=T))

### Paired Comparison of Likelihoods Within Model

In [ ]:
%%R
#precompute ROCs
levdist_rocs = list()
for (model_name in overview_set){
    one_model = subset(scores_across_time_short, model == model_name & likelihood_type == 'Levenshtein Distance')
    levdist_rocs[[model_name]] = roc(one_model$set_binarized, one_model$posterior_entropy)
}

In [ ]:
%%R
paired_comparison =  do.call('rbind', lapply(overview_set, function(model_name){    
    roc1 = rocs[[model_name]]
    roc2 = levdist_rocs[[model_name]]
    rtest = roc.test(roc1, roc2, method="delong")    

    prob = round(rtest$p.value, 3)        
    stat = abs(round(rtest$statistic, 3))    
    return(data.frame(model_name, prob, stat))        
}))

paired_comparison

In [ ]:
%%R
print(auc(rocs[['UniformPrior']]))
print(auc(levdist_rocs[['UniformPrior']]))

# Analysis 2: Do the models find the same words as adults?

In [ ]:
%%R
unique(successes_across_time$token)[1:100]

## Prior and Posterior Table

In [ ]:
%%R
unique(successes_across_time$likelihood_type)

### WFST

In [ ]:
%%R
# note that we don't need to separate the prior computations by the likelihood type-- it's the prior!

prior_probs = aggregate(prior_probability ~ model, successes_across_time, function(x){mean(-1 * log(x, base = 2))})
posterior_probs = aggregate(posterior_probability ~ model, subset(successes_across_time, likelihood_type == 'WFST'), function(x){mean(-1 * log(x, base = 2))})
prior_probs = merge(prior_probs, posterior_probs)
    

subset(prior_probs, model %in% overview_set)

In [ ]:
%%R
posterior_probs_sd = aggregate(posterior_probability ~ model, subset(successes_across_time, likelihood_type == 'WFST'), function(x){sem(-1 * log(x, base = 2))})
names(posterior_probs_sd) = c('model', 'posterior_probability_sem')


prior_probs_sem = aggregate(prior_probability ~ model, subset(successes_across_time, likelihood_type == 'WFST'), function(x){sem(-1 * log(x, base = 2))})
names(prior_probs_sem) = c('model', 'prior_probability_sem')

prior_probs = merge(merge(prior_probs, posterior_probs_sd), prior_probs_sem)
prior_probs = prior_probs[order(prior_probs$prior_probability),]
prior_probs$prior_low = prior_probs$prior_probability - prior_probs$prior_probability_sem
prior_probs$prior_high = prior_probs$prior_probability + prior_probs$prior_probability_sem
prior_probs$posterior_low = prior_probs$posterior_probability - prior_probs$posterior_probability_sem
prior_probs$posterior_high = prior_probs$posterior_probability + prior_probs$posterior_probability_sem
prior_probs


In [ ]:
%%R
# try bootstrapping it instead
library(boot)

fc <- function(x,i ){
	return(mean(x[i]))
}

x = rnorm(1000)
print(x[1:10])
boot.ci(boot(x, fc, R=1000), type="basic")$basic[,4:5]

# get the 95% ci

In [ ]:
%%R
library('parallel')
fc <- function(x,i ){
	return(mean(x[i]))
}

wfst_df = subset(successes_across_time, likelihood_type == 'WFST')
df_split = split(wfst_df, wfst_df$model) 
results = do.call('rbind', mclapply(df_split, function(df){
    x = -1 * log(df$prior_probability, base = 2)
    ci = boot.ci(boot(x, fc, R=10), type="basic")$basic
    return(data.frame(model = unique(df$model), low=ci[4], high=ci[5]))
}, mc.cores=24))



In [ ]:
results

In [ ]:
%%R
posterior_mean_rankings = aggregate(posterior_rank ~ model + likelihood_type, successes_across_time, function(x){mean_posterior_rank = mean(x)})
names(posterior_mean_rankings) = c('model', 'likelihood_type', 'mean_posterior_rank')

prior_mean_rankings = aggregate(prior_rank ~ model  + likelihood_type, successes_across_time, function(x){mean_prior_rank = mean(x)})
names(prior_mean_rankings) = c('model', 'likelihood_type', 'mean_prior_rank')


posterior_median_rankings = aggregate(posterior_rank ~ model + likelihood_type, successes_across_time, function(x){median_posterior_rank = median(x)+1})
names(posterior_median_rankings) = c('model', 'likelihood_type', 'median_posterior_rank')
# 0-indexed coming from Python, so incrementing by one

prior_median_rankings = aggregate(prior_rank ~ model  + likelihood_type, successes_across_time, function(x){median_prior_rank = median(x)+1})
names(prior_median_rankings) = c('model', 'likelihood_type', 'median_prior_rank')
# 0-indexed coming from Pythong, so incrementing by one

rankings = merge(prior_mean_rankings, posterior_mean_rankings, by=c('model', 'likelihood_type'))

rankings = merge(rankings, posterior_median_rankings, by=c('model', 'likelihood_type'))
rankings = merge(rankings, prior_median_rankings, by=c('model', 'likelihood_type'))



rankings = rankings[order(rankings$mean_prior_rank),]
subset(rankings, model %in% overview_set)

In [ ]:
%%R
for_table = subset(merge(prior_probs, rankings, all.x=T), model %in% overview_set)
for_table
for_table$prior_probability = round(for_table$prior_probability, 2)
for_table$posterior_probability = round(for_table$posterior_probability, 2)
for_table$mean_prior_rank = round(for_table$mean_prior_rank, 2)
for_table$mean_posterior_rank = round(for_table$mean_posterior_rank, 2)
#for_table = for_table[order(for_table$prior_probability),]

table_for_writing =  subset(for_table, likelihood_type == "WFST")[,c('model','prior_probability','mean_prior_rank','posterior_probability','mean_posterior_rank')]
names(table_for_writing) = c('Model', 'Prior Surprisal (bits)', 'Mean Prior Rank', 'Posterior Surprisal (bits)', 'Mean Posterior Rank')
table_for_writing

In [ ]:
%%R
write.csv(for_table, '../../output/csv/surprisal_table.csv', row.names=F)

In [ ]:
%%R
nrow(for_table)

In [ ]:
%%R
as.character(subset(for_table, likelihood_type == 'WFST')$model)

In [ ]:
%%R
table_for_ordering = subset(for_table, likelihood_type == 'WFST')
table_for_ordering = table_for_ordering[order(table_for_ordering$prior_probability),]

for_table$model  = factor(for_table$model, levels = table_for_ordering$model)
for_table$prior_probability_bits = paste(for_table$prior_probability, 'bits')

p1 =  ggplot(subset(for_table, likelihood_type == 'WFST')) + geom_bar(aes(x=model, y=prior_probability, fill=model), stat="identity"
) + coord_flip() + ylab('Average Prior Surprisal (bits)') + xlab('Prior') + theme_classic() +
theme(legend.position="none") + geom_errorbar(aes(x=model, ymin=prior_low, ymax = prior_high), width = 0
)+  geom_text(aes(x=model, y=prior_probability, label=prior_probability_bits), hjust=1.2, size=6
) 
print(p1)

ggsave('../../output/figures/prior_bar_chart.pdf')

In [ ]:
%%R
for_table$posterior_probability_bits = paste(for_table$posterior_probability, 'bits')

p1 = ggplot(subset(for_table, likelihood_type == 'WFST')) + geom_bar(aes(x=model, y=posterior_probability, fill=model), stat="identity"
)  + ylab('Average Posterior Surprisal (bits)') + xlab('Posterior') + theme_classic() +
theme(legend.position="none") + geom_errorbar(aes(x=model, ymin=posterior_low, ymax = posterior_high), width = 0
)+  geom_text(aes(x=model, y=posterior_probability, label=posterior_probability_bits), hjust=-.2, size=6
)  + coord_flip(ylim=c(0, 6))

print(p1)
ggsave('../../output/figures/posterior_bar_chart.pdf')

In [ ]:
%%R

p1 = ggplot(subset(for_table, likelihood_type == 'WFST')) + geom_bar(aes(x=model, y=mean_prior_rank, fill=model), stat="identity"
)  + ylab("Mean Prior Rank of Annotator's Transcription (out of 7,997 word types)") + xlab('Prior') + theme_classic() +
theme(legend.position="none") + geom_text(aes(x=model, y=mean_prior_rank, label=mean_prior_rank), hjust=-.4, size=6
)  + coord_flip(ylim =c(0, 5000)) 
print(p1)
ggsave('../../output/figures/prior_rank.pdf')


In [ ]:
%%R

p1 = ggplot(subset(for_table, likelihood_type == 'WFST')) + geom_bar(aes(x=model, y=mean_posterior_rank, fill=model), stat="identity"
)  + ylab("Mean Posterior Rank of Annotator's Transcription (out of 7,997 word types)") + xlab('Posterior') + theme_classic() +
theme(legend.position="none") + geom_text(aes(x=model, y=mean_posterior_rank, label=mean_posterior_rank), hjust=-.4, size=6
)  + coord_flip(ylim =c(0, 75)) 
print(p1)
ggsave('../../output/figures/posterior_rank.pdf')

### Compare Likelihood Functions

In [ ]:
%%R
library(ggpattern)

p1 = ggplot(for_table) + geom_bar(aes(x=model, y=mean_posterior_rank, fill=model, alpha=likelihood_type), stat="identity", position="dodge"
)  + ylab("Mean Posterior Rank of Annotator's Transcription (out of 7,997 word types)") + xlab('Prior') + theme_classic() +
theme(legend.position="none") + geom_text(data=subset(for_table, likelihood_type =='WFST'), aes(x=model, y=mean_posterior_rank, 
label=mean_posterior_rank), hjust=-.4, size=6, vjust=-.2,                                          
position = "dodge") + geom_text(data=subset(for_table, likelihood_type =='Levenshtein Distance'), aes(x=model, y=mean_posterior_rank, 
label=mean_posterior_rank), hjust=-.4, size=6, vjust=1.5,                                          
position = "dodge")  + coord_flip(ylim =c(0, 200)) 
print(p1)
ggsave('../../output/figures/posterior_rank_by_likelihood.pdf')

### Median Ranks

In [ ]:
%%R
# aggregate number by rank by mode
number_by_rank_by_model = aggregate(posterior_probability ~ posterior_rank + model,
    subset(successes_across_time, likelihood_type == 'WFST') , length) 
names(number_by_rank_by_model) = c('posterior_rank' , 'model', 'number_items')

number_by_rank_by_model = subset(number_by_rank_by_model, model %in% overview_set)
number_by_rank_by_model$model  = factor(number_by_rank_by_model$model, levels = table_for_ordering$model)


In [ ]:
%%R
p1 = ggplot(number_by_rank_by_model) + geom_line(aes(
 x=posterior_rank, y = number_items, color=model))  + theme_classic(
) + ylab('Number of Vocalizations') + xlab('Posterior Rank Under Model of Transcriber Word'
) + theme(legend.position = c(.5, .5)) + labs(color='Model')
ggsave('../../output/figures/density_posterior_rank_full.pdf')
print(p1)

In [ ]:
%%R
p1 = ggplot(subset(number_by_rank_by_model,
posterior_rank <= 9)) + geom_line(aes(
 x=posterior_rank+1, y = number_items, color=model))  + theme_classic(
) + ylab('Number of Vocalizations') + xlab('Posterior Rank Under Model of Transcriber Word'
) + theme(legend.position = c(.5, .5)) + labs(color='Model') + scale_x_continuous(
    breaks = c(1:10), labels = c(1:10))
ggsave('../../output/figures/density_posterior_rank_10.pdf')
print(p1)

In [ ]:
%%R

p1 = ggplot(subset(for_table, likelihood_type == 'WFST')) + geom_bar(aes(x=model, y=median_prior_rank, fill=model), stat="identity"
)  + ylab("Median Rank of Annotator's Transcription (out of 7,997 word types)") + xlab('Prior') + theme_classic() +
theme(legend.position="none") + geom_text(aes(x=model, y=median_prior_rank, label=median_prior_rank), hjust=-.4, size=6
)  + coord_flip(ylim =c(0, 5000)) 
print(p1)
ggsave('../../output/figures/median_prior_rank.pdf')



In [ ]:
%%R

p1 = ggplot(subset(for_table, likelihood_type == 'WFST')) + geom_bar(aes(x=model, y=median_posterior_rank, fill=model), stat="identity"
)  + ylab("Median Posterior Rank of Annotator's Transcription (out of 7,997 word types)") + xlab('Prior') + theme_classic() +
theme(legend.position="none") + geom_text(aes(x=model, y=median_posterior_rank, label=median_posterior_rank), hjust=-.4, size=6
)  + coord_flip(ylim =c(0, 75)) 
print(p1)
ggsave('../../output/figures/median_posterior_rank.pdf')

### Levenshtein Distance (Posteriors Only)

In [ ]:
%%R
lev_probs = aggregate(posterior_probability ~ model, subset(successes_across_time, likelihood_type == 'Levenshtein Distance'), function(x){mean(-1 * log(x, base = 2))})
lev_rankings = aggregate(posterior_rank ~ model, subset(successes_across_time, likelihood_type == 'Levenshtein Distance'), function(x){mean_posterior_rank = mean(x)})
names(lev_rankings) = c('model', 'mean_posterior_rank')
lev_for_table = subset(merge(lev_probs, lev_rankings, all.x=T), model %in% overview_set)
lev_for_table[order(lev_for_table$posterior_probability),]

In [ ]:
%%R
lev_for_table$posterior_probability = round(lev_for_table$posterior_probability, 2)
lev_for_table$mean_posterior_rank = round(lev_for_table$mean_posterior_rank, 2)
lev_for_table = lev_for_table[,c('model','posterior_probability','mean_posterior_rank')]
lev_for_table =  lev_for_table[order(lev_for_table$posterior_probability),]
names(lev_for_table) = c('Model', 'Posterior Surprisal (bits)', 'Mean Posterior Rank')
lev_for_table

In [ ]:
%%R

# 7/15/21 not a code reference, but debugging help:
# https://stackoverflow.com/questions/60083062/tidyrspread-error-each-row-of-output-must-be-identified-by-a-unique-combina
# https://community.rstudio.com/t/spread-error-each-row-of-output-must-be-identified-by-a-unique-combination-of-keys/88325
# https://garrettgman.github.io/tidying

successes_across_time_wide =  spread(successes_across_time[,c('model','prior_probability',
'bert_token_id','likelihood_type')], model, prior_probability)
successes_across_time_wide = subset(successes_across_time_wide, likelihood_type == 'WFST')
head(successes_across_time_wide)

In [ ]:
%%R

model_names = overview_set
print('Model names:')
print(model_names)

for (model_name in model_names){
    successes_across_time_wide[[model_name]] = as.numeric(successes_across_time_wide[[model_name]])
}

successes_across_time_wide_nona = drop_na(successes_across_time_wide)
print(dim(successes_across_time_wide_nona))

In [ ]:
%%R
unique(successes_across_time_wide_nona$likelihood_type)

### t-test Pairwise Model Comparison for WFST

In [ ]:
%%R
#Pairwise paired t-test between all of the models with WFST only

rvs = mat.or.vec(length(model_names), length(model_names))

for (model_idx in c(1:length(model_names))){
    for (comparison_model_idx in c(1:length(model_names))){
        if (model_idx == comparison_model_idx){
            rvs[model_idx, comparison_model_idx] = NA
        } else {
            model_name = model_names[model_idx]
            comparison_model_name = model_names[comparison_model_idx]
            rval = t.test(
                successes_across_time_wide_nona[[model_name]], 
                successes_across_time_wide_nona[[comparison_model_name]],
              paired=T)$p.value
            rvs[model_idx, comparison_model_idx] = rval
        }
    }
}
rvs = data.frame(rvs)

rownames(rvs) = model_names
colnames(rvs) = model_names

rvs

### Prior Surprisal Across Ages Plots

In [ ]:
%R mean_surprisal_by_age = aggregate(prior_probability ~ age + model + likelihood_type, successes_across_time ,\
                function(x){mean(-1 * log(x, base =2))})
%R names(mean_surprisal_by_age) = c('age','model', 'likelihood_type','prior_surprisal_mean')
%R mean_surprisal_by_age_sem = aggregate(prior_probability ~ age + model + likelihood_type, \
        successes_across_time, function(x){sem(-1 * log(x, base = 2))})
%R names(mean_surprisal_by_age_sem) = c('age','model','likelihood_type','prior_surprisal_sem')
%R mean_surprisal_by_age = merge(mean_surprisal_by_age, mean_surprisal_by_age_sem) 

In [ ]:
%%R -w 1000 -h 500

overview_mean_surprisal_by_age <- filter_model_subset(mean_surprisal_by_age, overview_set)
    
overview_mean_surprisal_for_gray = do.call('rbind', lapply(unique(overview_mean_surprisal_by_age$model), function(model){ get_ref_df(model, overview_mean_surprisal_by_age)}))

ggplot(overview_mean_surprisal_by_age) + geom_point(aes(x=age, y=prior_surprisal_mean, color=model,
    shape = model)) + geom_line(aes(x=age, y=prior_surprisal_mean, color=model)
    ) + geom_errorbar(aes(x=age, ymin=prior_surprisal_mean - .5*prior_surprisal_sem, 
    ymax=prior_surprisal_mean + .5*prior_surprisal_sem, color=model ), width=0) + theme_classic(
    base_size = 24) + ylab('Average Prior Surprisal') + xlab('Child Age in Years') +
    facet_wrap(~model) + geom_line(data=overview_mean_surprisal_for_gray, aes(x=age,y=prior_surprisal_mean, linetype = line_model),colour='gray', alpha=1) +
    theme(legend.position='none') + scale_shape_manual(values=1:nlevels(overview_mean_surprisal_by_age$model)
    ) + scale_color_discrete()

# # Best performing CDL Context all/all, CHILDES 1gram and Uniform Prior 

In [ ]:
%%R -w 1000 -h 500

subset_mean_surprisal_by_age <- filter_model_subset(mean_surprisal_by_age, splits_tags_set)
    
subset_mean_surprisal_for_gray = do.call('rbind', lapply(unique(subset_mean_surprisal_by_age$model), function(model){ get_ref_df(model, subset_mean_surprisal_by_age)}))

ggplot(subset_mean_surprisal_by_age) + geom_point(aes(x=age, y=prior_surprisal_mean, color=model,
    shape = model)) + geom_line(aes(x=age, y=prior_surprisal_mean, color=model)
    ) + geom_errorbar(aes(x=age, ymin=prior_surprisal_mean - .5*prior_surprisal_sem, 
    ymax=prior_surprisal_mean + .5*prior_surprisal_sem, color=model ), width=0) + theme_classic(
    base_size = 24) + ylab('Average Prior Surprisal') + xlab('Child Age in Years') +
    facet_wrap(~model) + geom_line(data=subset_mean_surprisal_for_gray, aes(x=age,y=prior_surprisal_mean, linetype = line_model),colour='gray', alpha=1) +
    theme(legend.position='none')



## Posterior

In [ ]:
%R mean_surprisal_by_age = aggregate(posterior_probability ~ age + model + likelihood_type, successes_across_time ,\
                function(x){mean(-1 * log(x, base = 2))})
%R names(mean_surprisal_by_age) = c('age','model','likelihood','posterior_surprisal_mean')
%R mean_surprisal_by_age_sem = aggregate(posterior_probability ~ age + model + likelihood_type, \
        successes_across_time, function(x){sem(-1 * log(x, base =2))})
%R names(mean_surprisal_by_age_sem) = c('age','model','likelihood','posterior_surprisal_sem')
%R mean_surprisal_by_age = merge(mean_surprisal_by_age, mean_surprisal_by_age_sem) 
#%R print(mean_surprisal_by_age)

In [ ]:
%%R -w 600 -h 800

overview_mean_surprisal_by_age <- filter_model_subset(mean_surprisal_by_age, overview_set)
    
overview_mean_surprisal_for_gray = do.call('rbind', lapply(unique(overview_mean_surprisal_by_age$model), function(model){ get_ref_df(model, overview_mean_surprisal_by_age)}))

ggplot(overview_mean_surprisal_by_age) + geom_point(aes(x=age, y=posterior_surprisal_mean, color=model,
    shape = model)) + geom_line(aes(x=age, y=posterior_surprisal_mean, color=model)
    ) + geom_errorbar(aes(x=age, ymin=posterior_surprisal_mean - .5*posterior_surprisal_sem, 
    ymax=posterior_surprisal_mean + .5*posterior_surprisal_sem, color=model ), width=0) + theme_classic(
    base_size = 24) + ylab('Average Posterior Surprisal') + xlab('Child Age in Years') +
    facet_grid(model~likelihood) + geom_line(data=overview_mean_surprisal_for_gray, aes(x=age,y=posterior_surprisal_mean, linetype = line_model),colour='gray', alpha=1) +
    theme(legend.position='none')


In [ ]:
%%R -w 1000 -h 500

subset_mean_surprisal_by_age <- filter_model_subset(mean_surprisal_by_age, splits_tags_set)
    
subset_mean_surprisal_for_gray = do.call('rbind', lapply(unique(subset_mean_surprisal_by_age$model), function(model){ get_ref_df(model, subset_mean_surprisal_by_age)}))

ggplot(subset_mean_surprisal_by_age) + geom_point(aes(x=age, y=posterior_surprisal_mean, color=model,
    shape = model)) + geom_line(aes(x=age, y=posterior_surprisal_mean, color=model)
    ) + geom_errorbar(aes(x=age, ymin=posterior_surprisal_mean - .5*posterior_surprisal_sem, 
    ymax=posterior_surprisal_mean + .5*posterior_surprisal_sem, color=model ), width=0) + theme_classic(
    base_size = 24) + ylab('Average Posterior Surprisal') + xlab('Child Age in Years') +
    facet_wrap(likelihood~model) + geom_line(data=subset_mean_surprisal_for_gray, aes(x=age,y=posterior_surprisal_mean, linetype = line_model),colour='gray', alpha=1) +
    theme(legend.position='none')


In [ ]:
%%R
unique(as.character(mean_surprisal_by_age$model))

In [ ]:
%%R
mean_surprisal_by_age$model_shortname = gsub('\\+Tags', '', mean_surprisal_by_age$model)

In [ ]:
%%R -w 1000 -h 500

selected_models = c('CDL+Younger+Context+Tags', 'CDL+Older+Context+Tags', 'CDL+Context+Tags','Switchboard+Context')

p1 =  ggplot(subset(mean_surprisal_by_age, likelihood == 'WFST' &  model %in% selected_models)
    ) + geom_errorbar(aes(x=age, ymin=posterior_surprisal_mean - posterior_surprisal_sem, 
    ymax=posterior_surprisal_mean + posterior_surprisal_sem), color="gray",  width=0) + geom_point(aes(x=age, y=posterior_surprisal_mean, color=model_shortname,
    shape = model_shortname), size=5) + geom_line(aes(x=age, y=posterior_surprisal_mean, color=model_shortname)
    )  + theme_classic(
    base_size = 24) + ylab('Average Posterior Surprisal') + xlab('Child Age in Years') + theme(
    legend.position=c(.6, .8))  +  guides(shape = guide_legend(title = "Model"), color = guide_legend(title = "Model"))

ggsave('../../output/figures/posterior_surprisal_by_age.pdf', height=6, width=8)
p1

In [ ]:
scores_across_time_short.drop_duplicates(subset=['bert_token_id']).set.value_counts()

### Posterior Probability by Edit Distance

In [ ]:
%%R
# have to do some data rotation to handle the fact that each observation needs both a Levenshtein distance and an edit dsitance
levdists = subset(successes_across_time, likelihood_type == "Levenshtein Distance" & model %in% overview_set)
dim(levdists)
wfsts = subset(successes_across_time, likelihood_type == "WFST" & model %in% overview_set)
dim(wfsts)

In [ ]:
%%R
unique(as.character(levdists$model))

In [ ]:
%%R
wfsts$wfst_distance = wfsts$edit_distance # copy wfst distance into its own column
wfsts$edit_distance = NULL
levdists$wfst_distance = 0 # leave this column unpopulated on levdists

In [ ]:
%%R
print(dim(wfsts))
print(dim(levdists))

In [ ]:
%%R
wfsts_merged = merge(wfsts, levdists[,c('bert_token_id','edit_distance')]) # this takes a while

In [ ]:
%%R
levdists[,c('bert_token_id','edit_distance')][1:10,]

In [ ]:
%%R 
ed = rbind.fill(wfsts_merged, levdists) # this takes a while

In [ ]:
%%R
# confirm that edit distances are integers
unique(ed$edit_distance)

#### Statistical Test for Surprisal by Edit Distance

In [ ]:
%%R
names(ed)

In [ ]:
%%R
library(lme4)
names(ed)
lm1 = lm(posterior_probability ~ edit_distance , ed)
lm2 = lm(posterior_probability ~ edit_distance * model , ed)


print(BIC(lm1))
print(BIC(lm2))

# is a particular model actually lower? 

In [ ]:
%%R
surprisal_by_edit_distance = aggregate(posterior_probability ~ edit_distance + model + likelihood_type, ed, 
    function(x){mean(-1 * log(x, base = 2))})
names(surprisal_by_edit_distance) = c('edit_distance', 'model', 'likelihood_type', 'mean_posterior_surprisal')
sem <- function(x) sd(x)/sqrt(length(x))
surprisal_by_edit_distance_sem = aggregate(posterior_probability ~ edit_distance + model + likelihood_type, ed, 
    function(x){sem(-1 * log(x, base = 2))})
names(surprisal_by_edit_distance_sem) = c('edit_distance','model','likelihood_type','mean_posterior_surprisal_sem')
surprisal_by_edit_distance = merge(surprisal_by_edit_distance, surprisal_by_edit_distance_sem)
surprisal_by_edit_distance$high = surprisal_by_edit_distance$mean_posterior_surprisal + surprisal_by_edit_distance$mean_posterior_surprisal_sem 
surprisal_by_edit_distance$low = surprisal_by_edit_distance$mean_posterior_surprisal - surprisal_by_edit_distance$mean_posterior_surprisal_sem



In [ ]:
%%R
models_for_8_plots

In [ ]:
%%R

surprisal_by_edit_distance_for_plot = subset(surprisal_by_edit_distance, likelihood_type != 'WFST' & edit_distance <= 6)


surprisal_for_gray = do.call('rbind', lapply(unique(surprisal_by_edit_distance_for_plot$model), function(model){ get_ref_df(model, surprisal_by_edit_distance_for_plot)}))

p1= ggplot(surprisal_by_edit_distance_for_plot) + geom_point(aes(x=edit_distance, 
    y = mean_posterior_surprisal, color = model, shape=model), size=5) + theme_classic(base_size=20) + xlab( 
    'Edit Distance from Actual to Citation Form for Word') + ylab('Average Posterior Surprisal (bits)') + 
    geom_errorbar(aes(x=edit_distance, ymin=low, ymax= high, color=model), width=0) + 
    geom_line(aes(x=edit_distance, y = mean_posterior_surprisal, color = model, 
shape=model)) + theme(legend.position=c(.3, .8), legend.background=element_blank(), legend.title = element_blank()) + scale_shape_discrete(name  ="Model" 
) + scale_color_discrete(name  ="Model") + annotate("text", label="Lower is better", 
    x=5.5, y=.05, alpha=.5, size=6, fontface ="italic")  + scale_x_continuous(breaks=c(0:6), labels = c(0:6)
) + scale_shape_manual(values=1:nlevels(surprisal_by_edit_distance_for_plot$model)) + scale_color_manual(
        values=1:nlevels(surprisal_by_edit_distance_for_plot$model))

print(p1)

In [ ]:
%%R
surprisal_by_edit_distance_for_plot = subset(surprisal_by_edit_distance, likelihood_type == 'WFST' & edit_distance <= 6)


surprisal_for_gray = do.call('rbind', lapply(unique(surprisal_by_edit_distance_for_plot$model), function(model){ get_ref_df(model, surprisal_by_edit_distance_for_plot)}))

p1= ggplot(surprisal_by_edit_distance_for_plot) + geom_point(aes(x=edit_distance, 
    y = mean_posterior_surprisal, color = model, shape=model), size=5) + theme_classic(base_size=20) + xlab( 
    'Edit Distance from Actual to Citation Form for Word') + ylab('Average Posterior Surprisal (bits)') + 
    geom_errorbar(aes(x=edit_distance, ymin=low, ymax= high, color=model), width = 0) + 
    geom_line(aes(x=edit_distance, y = mean_posterior_surprisal, color = model, 
shape=model, linetype=likelihood_type)) + theme(legend.position=c(.3, .8), legend.background=element_blank()) + scale_shape_discrete(name  ="Model" 
) + scale_color_discrete(name  ="Model") + annotate("text", label="Lower is better", 
    x=5.5, y=.05, alpha=.5, size=6, fontface ="italic") +
facet_wrap(~model) + geom_line(data=surprisal_for_gray, aes(x=edit_distance,y=mean_posterior_surprisal, linetype = line_model),colour='gray', alpha=1)+
theme(legend.position='none')
print(p1)

#### Levenshtein Distance Plots

In [ ]:
%%R
# Multiplier for each 
unigram_prior = subset(surprisal_by_edit_distance, model == "UniformPrior" & likelihood_type == 'Levenshtein Distance')[,c('edit_distance',
    'mean_posterior_surprisal')]
names(unigram_prior)[2] = "unigram_prior_prob"

surprisal_by_edit_distance_lev = merge(subset(surprisal_by_edit_distance, likelihood_type == 'Levenshtein Distance'), unigram_prior)
surprisal_by_edit_distance_lev$multiplier = 2^( surprisal_by_edit_distance_lev$unigram_prior_prob -
    surprisal_by_edit_distance_lev$mean_posterior_surprisal)

head(surprisal_by_edit_distance_lev, n=10)


In [ ]:
%%R

# Added: no subsetting for the odds uniform prior

p2= ggplot(subset(surprisal_by_edit_distance_lev, edit_distance <= 6 & model %in% overview_set 
& likelihood_type =='Levenshtein Distance' & model != 'UniformPrior')) + geom_point(aes(x=edit_distance, 
    y = multiplier, color = model, shape=model), size=5) + theme_classic(base_size=20) + xlab( 
    'Edit Distance from Actual to Citation Form for Word') + ylab(
    'Odds vs. Uniform Prior') + geom_line(aes(x=edit_distance, 
    y = multiplier, color = model, shape=model))+ theme(legend.position=c(.2, .8), 
    legend.background=element_blank(), legend.title = element_blank()) + scale_shape_discrete(name  ="Model" 
) + scale_color_discrete(name  ="Model") + annotate("text", label="Higher is better", 
    x=5.5, y=512, alpha=.5, size=6, fontface ="italic") + scale_y_continuous(trans='log2'
) + scale_shape_manual(values = 1:8) + scale_color_manual(values = 1:8)    
print(p2)

#### WFST Plots

In [ ]:
%%R
# make sure it uses the same colors
surprisal_by_edit_distance_for_plot = subset(surprisal_by_edit_distance, likelihood_type == 'WFST' & edit_distance <= 6)
surprisal_by_edit_distance_for_plot$model_shortname= gsub('\\+Tags','',surprisal_by_edit_distance_for_plot$model)
surprisal_by_edit_distance_for_plot$model_shortname = factor(surprisal_by_edit_distance_for_plot$model_shortname, levels = models_for_8_plots)


In [ ]:
%%R

surprisal_by_edit_distance_for_plot = subset(surprisal_by_edit_distance, likelihood_type == 'WFST' & edit_distance <= 6)


surprisal_for_gray = do.call('rbind', lapply(unique(surprisal_by_edit_distance_for_plot$model), function(model){ get_ref_df(model, surprisal_by_edit_distance_for_plot)}))

p1= ggplot(surprisal_by_edit_distance_for_plot)  + theme_classic(base_size=20) + xlab( 
    'Edit Distance from Actual to Citation Form for Word') + ylab('Average Posterior Surprisal (bits)') + 
    geom_line(aes(x=edit_distance, y = mean_posterior_surprisal, color = model, 
shape=model)) + theme(legend.position=c(.3, .8), legend.background=element_blank(), legend.title = element_blank()) + scale_shape_discrete(name  ="Model" 
) + scale_color_discrete(name  ="Model") + annotate("text", label="Lower is better", 
    x=5.5, y=.05, alpha=.5, size=6, fontface ="italic")  + scale_x_continuous(breaks=c(0:6), labels = c(0:6)
) + geom_point(aes(x=edit_distance, 
    y = mean_posterior_surprisal, color = model, shape=model), size=5) + scale_shape_manual(values=1:nlevels(surprisal_by_edit_distance_for_plot$model)) + scale_color_manual(
        values=1:nlevels(surprisal_by_edit_distance_for_plot$model)) + scale_color_manual(
        values=colors_for_8_plots) + scale_shape_manual(values = shapes_for_8_plots) + 
    geom_errorbar(aes(x=edit_distance, ymin=low, ymax= high), color="gray", width=0)


ggsave('../../output/figures/surpsisal_by_edit_distance_wfst.pdf', height=6, width=8)    
print(p1)

### Proportion Agreeing with Annotator

In [ ]:
%%R
proportion_retrieved = aggregate(prior_rank ~ model + likelihood_type, successes_across_time, function(x){mean(x < 5)})
proportion_retrieved

In [ ]:
%%R
proportion_retrieved = aggregate(posterior_rank ~ model + likelihood_type, successes_across_time, function(x){mean(x == 0)})
proportion_retrieved

### Proportion Agreeing with Annotator Per Age

In [ ]:
%%R
names(successes_across_time)

In [ ]:
%%R
proportion_retrieved = aggregate(posterior_rank ~ model + age + likelihood_type, successes_across_time, function(x){mean(x == 0)})

In [ ]:
%%R -w 750 -h 500
ggplot(filter_model_subset(subset(proportion_retrieved, likelihood_type == 'WFST'), overview_set)) + geom_line(aes(x=age, y=posterior_rank, color=model)
) + geom_point(aes(x=age, y=posterior_rank, color=model, shape=model)) + coord_cartesian(ylim=c(0,1)) + xlab('Child Age') + ylab(
    'Proportion of Top-1 Guesses\n Agreeing with Annotator') + theme_classic(base_size=28) + theme(legend.background = element_blank())




### Posterior by Edit Distance by Likelihood 

In [ ]:
from src.utils import load_splits
all_tokens_phono = load_splits.load_phono()




In [ ]:
all_tokens_phono.columns

In [ ]:
all_tokens_phono['citation_len'] = [len(x) for x in all_tokens_phono.model_phonology_no_dia]
all_tokens_phono['child_len'] = [len(x) for x in all_tokens_phono.actual_phonology_no_dia]
all_tokens_phono['phono_change'] = all_tokens_phono['child_len'] - all_tokens_phono['citation_len']
ipa_tokens = all_tokens_phono.dropna(subset=['actual_phonology_no_dia'])[['bert_token_id','phono_change']]
%R -i ipa_tokens

In [ ]:
%%R
ed_with_phono_change = merge(ed, ipa_tokens)

In [ ]:
%%R
names(ed_with_phono_change)

In [ ]:
%%R

#need the length of the adult string vs. the child string 

prob_by_edit_distance_likelihood = aggregate(posterior_probability ~ likelihood_type + phono_change + model, ed_with_phono_change, mean)

prob_by_edit_distance_likelihood_sem = aggregate(posterior_probability ~ likelihood_type + phono_change + model, ed_with_phono_change, sem)
names(prob_by_edit_distance_likelihood_sem) = c('likelihood_type', 'phono_change', 'model', 'sem')

prob_by_edit_distance_likelihood = merge(prob_by_edit_distance_likelihood, prob_by_edit_distance_likelihood_sem)
prob_by_edit_distance_likelihood$high = prob_by_edit_distance_likelihood$posterior_probability + prob_by_edit_distance_likelihood$sem
prob_by_edit_distance_likelihood$low = prob_by_edit_distance_likelihood$posterior_probability - prob_by_edit_distance_likelihood$sem


In [ ]:
%%R
prob_by_edit_distance_likelihood$likelihood_type = factor(prob_by_edit_distance_likelihood$likelihood_type, 
levels = c('WFST', 'Levenshtein Distance'))

In [ ]:
%%R
colors = 


p1 = ggplot(subset(prob_by_edit_distance_likelihood, model == "CDL+Context+Tags")) + geom_errorbar(
aes(x=phono_change, ymin = low, ymax=high), color='gray', width=0) + geom_line(aes(x=phono_change, y=posterior_probability,
    color = likelihood_type, linetype = likelihood_type )) + geom_point(aes(x=phono_change, y=posterior_probability,
    color = likelihood_type, shape=likelihood_type), size=3) + theme_classic() + ylab('Average Posterior Probability of Transcriber Interpretations') + xlab (
    '← Child Omits Phonemes — | — Child Adds Phonemes →') + geom_vline(xintercept=0, linetype='dashed'
) + scale_x_continuous(breaks=c(-5:5), labels = c(-5:5))  + scale_shape_discrete(name = "Likelihood Function") + scale_linetype_discrete(name = "Likelihood Function") + theme(legend.position = c(.8, .2)
) + annotate("text", label="Higher is better", 
    x=2.5, y=1, alpha=.5, size=6, fontface ="italic")  + scale_color_manual(values=c("#6f80d6", "#E69F00"), name = "Likelihood Function")

ggsave("../../output/figures/likelihood_function_by_change.png", width = 5, height=5)
print(p1)

# Child-Specific Analyses

In [ ]:

all_tokens_phono_short = all_tokens_phono[['bert_token_id','target_child_name']]
%R -i all_tokens_phono_short

In [ ]:
%%R
successes_across_time_with_child_label = merge(successes_across_time, all_tokens_phono_short)

In [ ]:
%%R
names(successes_across_time_with_child_label)

In [ ]:
%%R
posterior_entropy_by_child = aggregate(posterior_entropy ~ model + likelihood_type + target_child_name, successes_across_time_with_child_label, mean)
subset(posterior_entropy_by_child, likelihood_type == 'WFST' & model == 'CDL+Context+Tags')

In [ ]:
%%R
proportion_retrieved = aggregate(prior_probability ~ model  + likelihood_type + target_child_name, successes_across_time_with_child_label, function(x){mean(-1 *log(x, base=2))})
subset(proportion_retrieved, likelihood_type == 'WFST' & model == 'CDL+Context+Tags')

In [ ]:
%%R
proportion_retrieved = aggregate(posterior_rank ~ model  + likelihood_type + target_child_name, successes_across_time_with_child_label, function(x){mean(x == 0)})
names(proportion_retrieved)[4] = 'proportion_top_1'
subset(proportion_retrieved, likelihood_type == 'WFST' & model == 'CDL+Context+Tags')


In [ ]:
%%R
proportion_retrieved = aggregate(prior_rank ~ model  + likelihood_type + target_child_name, successes_across_time_with_child_label, function(x){mean(x == 0)})
names(proportion_retrieved)[4] = 'proportion_top_1'
subset(proportion_retrieved, likelihood_type == 'WFST' & model == 'CDL+Context+Tags')

